In [3]:
!export XDG_CACHE_HOME=/datasets/home/90/490/$(whoami)/.cache

In [ ]:
!pip install newsapi-python --user
from newsapi import NewsApiClient

sources = 'bbc-news, fortune, google-news, hacker-news, techradar, techcrunch, the-wall-street-journal, wired'
newsapi = NewsApiClient(api_key='YourToken')
top_headlines = newsapi.get_everything(q='AI', sources=sources, language='en')

In [44]:
import nltk
from nltk.corpus import wordnet as wn

In [45]:
# could not get pip working for the newsapi so just did it locally then went back to datahub for nltk
titles = ['Daily Crunch: Twitter acquires Fabula AI',
 "This AI Uses Echolocation to Identify What You're Doing",
 'When will customers start buying all those AI chips?',
 'Getaround, Facebook, AI chips, Nvidia, Africa, and immigration',
 'Huawei launches AI-backed database to target enterprise customers',
 'Robotics startups won’t win without also incorporating AI, as Karakuri’s fundraise shows',
 "Inside Facebook's New Robotics Lab, Where AI and Machines Friend One Another",
 'Pearl, the healthcare spinout from LA-based AI startup, GumGum, raises $11 million',
 'Amazon will use AI to help you shop for clothes with StyleSnap',
 'SentinelOne raises $120M for its fully-autonomous, AI-based endpoint security solution',
 'Self-driving delivery van startup Gatik AI comes out of stealth with Walmart partnership',
 'LG developed its own AI chip to make its smart home products even smarter',
 'MIT and U.S. Air Force team up to launch AI accelerator',
 'For less than $10, anyone can make an AI write a fake UN speech',
 'Alibaba-backed facial recognition startup Megvii raises $750 million',
 'Bill Gates, NEO, Gigafund backing Luminous in photonics supercomputer moonshot',
 'Google’s Latest Message: We’re Just Here to Help',
 'Verified Expert Brand Designer: Base',
 'Hailo launches its newest deep learning chip',
 'Sam Altman’s leap of faith']

In [59]:
qd =['alarm_clock',
  'ambulance',
  'angel',
  'ant',
  'antyoga',
  'backpack',
  'barn',
  'basket',
  'bear',
  'bee',
  'beeflower',
  'bicycle',
  'bird',
  'book',
  'brain',
  'bridge',
  'bulldozer',
  'bus',
  'butterfly',
  'cactus',
  'calendar',
  'castle',
  'cat',
  'catbus',
  'catpig',
  'chair',
  'couch',
  'crab',
  'crabchair',
  'crabrabbitfacepig',
  'cruise_ship',
  'diving_board',
  'dog',
  'dogbunny',
  'dolphin',
  'duck',
  'elephant',
  'elephantpig',
  'eye',
  'face',
  'fan',
  'fire_hydrant',
  'firetruck',
  'flamingo',
  'flower',
  'floweryoga',
  'frog',
  'frogsofa',
  'garden',
  'hand',
  'hedgeberry',
  'hedgehog',
  'helicopter',
  'kangaroo',
  'key',
  'lantern',
  'lighthouse',
  'lion',
  'lionsheep',
  'lobster',
  'map',
  'mermaid',
  'monapassport',
  'monkey',
  'mosquito',
  'octopus',
  'owl',
  'paintbrush',
  'palm_tree',
  'parrot',
  'passport',
  'peas',
  'penguin',
  'pig',
  'pigsheep',
  'pineapple',
  'pool',
  'postcard',
  'power_outlet',
  'rabbit',
  'rabbitturtle',
  'radio',
  'radioface',
  'rain',
  'rhinoceros',
  'rifle',
  'roller_coaster',
  'sandwich',
  'scorpion',
  'sea_turtle',
  'sheep',
  'skull',
  'snail',
  'snowflake',
  'speedboat',
  'spider',
  'squirrel',
  'steak',
  'stove',
  'strawberry',
  'swan',
  'swing_set',
  'the_mona_lisa',
  'tiger',
  'toothbrush',
  'toothpaste',
  'tractor',
  'trombone',
  'truck',
  'whale',
  'windmill',
  'yoga',
  'yogabicycle',
  'everything']

In [60]:
#discarding quickdraw classes containing more than one word
cleanQD = []
for w in qd:
    if len(w.split(' ')) > 1:
        print("discarding: " + w)
    else:
        cleanQD.append(w)

In [61]:
#discarding quickdraw classes that do not have synsets (needed for comparison)
synQD = []
for w in cleanQD:
    if len(wn.synsets(w)) > 0:
        synQD.append(wn.synsets(w)[0])
    else:
        print("discarding: " + w)


discarding: antyoga
discarding: beeflower
discarding: catbus
discarding: catpig
discarding: crabchair
discarding: crabrabbitfacepig
discarding: dogbunny
discarding: elephantpig
discarding: firetruck
discarding: floweryoga
discarding: frogsofa
discarding: hedgeberry
discarding: lionsheep
discarding: monapassport
discarding: pigsheep
discarding: power_outlet
discarding: rabbitturtle
discarding: radioface
discarding: swing_set
discarding: the_mona_lisa
discarding: yogabicycle
discarding: everything


In [63]:
print("remaining sketchRNN classes: %d" % len(synQD) )

remaining sketchRNN classes: 92


In [64]:
qdTitles = []
for j in range(len(titles)):
    titleSN = []
    # creating synsets for each word in title
    for w in titles[j].split(' '):
        if len(wn.synsets(w)) > 0:
            titleSN.append(wn.synsets(w)[0])
    
    if(len(titleSN) == 0):
        continue
        
    # comparing each word in title by similarity to the quickdraw classes
    simWords = []
    for sn in titleSN:
        wordSim = [sn.path_similarity(s) for s in synQD]
        simWords.append(wordSim)
    
    simWords = np.array(simWords)
    if(len(simWords) == 0):
        continue
        
    # extracting most similar word to title word
    closestToTitle = []
    try:
        for i in range(len(titleSN)):
            qdCounterpart = [None, None]
            qdCounterpart[1] = qd[np.argmax(simWords[i])]
            qdCounterpart[0] = np.max(simWords[i])
            closestToTitle.append(qdCounterpart)
    except:
        continue
        
    # taking top 3 most similar
    closestToTitle.sort(reverse=True)
    qdTitles.append([titles[j], [s[1] for s in closestToTitle[:3]]])

In [65]:
qdTitles

[['Daily Crunch: Twitter acquires Fabula AI',
  ['passport', 'barn', 'butterfly']],
 ["This AI Uses Echolocation to Identify What You're Doing",
  ['passport', 'passport', 'butterfly']],
 ['Getaround, Facebook, AI chips, Nvidia, Africa, and immigration',
  ['butterfly', 'butterfly']],
 ['Huawei launches AI-backed database to target enterprise customers',
  ['pool', 'passport', 'lighthouse']],
 ['Pearl, the healthcare spinout from LA-based AI startup, GumGum, raises $11 million',
  ['palm_tree', 'butterfly', 'butterfly']],
 ['Amazon will use AI to help you shop for clothes with StyleSnap',
  ['passport', 'book', 'book']],
 ['SentinelOne raises $120M for its fully-autonomous, AI-based endpoint security solution',
  ['pineapple', 'palm_tree', 'angel']],
 ['Self-driving delivery van startup Gatik AI comes out of stealth with Walmart partnership',
  ['pineapple', 'butterfly', 'butterfly']],
 ['MIT and U.S. Air Force team up to launch AI accelerator',
  ['pool', 'book', 'pineapple']],
 ['For